In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


train = pd.read_csv('../../data/train.tsv', sep='\t')
test = pd.read_csv('../../data/test.tsv', sep='\t')
train.shape


/home/zhecan/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/zhecan/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [ ]:
# from sklearn.cross_validation import train_test_split
# train_train, train_test = train_test_split(train, test_size=0.3, random_state=42)
split_indexs = int(len(train)/3)
train_test = train[: split_indexs]
train_train = train[split_indexs:]


In [3]:
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
from textblob import TextBlob
def extract_verbs_and_adj(phrase):
    words = []
    
    tags = TextBlob(phrase).tags
    for tag in tags:
        if tag[1][:2] == "VB" or tag[1][:2] == "JJ":
            words.append(tag[0])
    return words

def looping_extract_verbs_and_adj(data_set):
    phrases_vb_adj = []

    for (index, phrase) in enumerate(data_set):
        if index % 10 == 0:
            print index
        phrases_vb_adj.append(' '.join(extract_verbs_and_adj(phrase)))
    return phrases_vb_adj

train_train_phrases_vb_adj = looping_extract_verbs_and_adj(train_train.Phrase)
train_test_phrases_vb_adj = looping_extract_verbs_and_adj(train_test.Phrase)
test_phrases_vb_adj = looping_extract_verbs_and_adj(test.Phrase)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670

In [ ]:
def add_vb_adj_sentiment(phrases_vb_adj):
    v2 = CountVectorizer(ngram_range=(0, 4),  min_df=2, max_df=0.95, max_features=1000)
    X2 = v2.fit_transform(phrases_vb_adj)
    print X2.shape

    sentiments = []
    for feature in v2.get_feature_names():
        sentiment = TextBlob(feature).sentiment
        value = sentiment.polarity * sentiment.subjectivity
        sentiments.append(value)

    print v2.get_feature_names()
    print sentiments

    from scipy.sparse import csr_matrix

    X3 = csr_matrix(X2.shape)
    for index in range(X2.shape[0]):
        row = X2[index]
        for col in range(row.shape[1]):
            X3[index, col] = X2[index, col] * sentiments[col]

    from scipy.sparse import hstack
    X4 = hstack([X1[:amount], X3])
    print X4.shape
    return X4

X_train = add_vb_adj_sentiment(train_phrases_vb_adj)
X_test = add_vb_adj_sentiment(test_phrases_vb_adj)

In [ ]:
from sklearn.svm import SVC
clf = SVC()
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)



clf.fit(X_train_train, y_train_train) 

y_train_test_predict = clf.predict(X_train_test)
y_test_predict = clf.predict(X_test)


train_output = pd.DataFrame({
    'PhraseId': train.PhraseId,
    'Phrase':  train_lemma_phrase
})



test_output = pd.DataFrame({
    'PhraseId': test.PhraseId,
    'Phrase':  test_lemma_phrase
})